In [10]:
from trubrics.context import DataContext
from trubrics.example import get_titanic_data_and_model
from trubrics.validations import ModelValidator, Trubric

In [11]:
_, test_df, model = get_titanic_data_and_model()

In [17]:
test_df.head()

,Sex,Embarked,Title,Pclass,Age,SibSp,Parch,Fare,Survived
751,male,S,Master,3,6.0,0,1,12.48,1
90,male,S,Mr,3,29.0,0,0,8.05,0
852,female,C,Miss,3,9.0,1,1,15.25,0
718,male,Q,Mr,3,NaN,0,0,15.50,0
36,male,C,Mr,3,NaN,0,0,7.23,1


In [22]:
model.predict(test_df)

array([1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [23]:
# 1. Init DataContext
data_context = DataContext(
    testing_data=test_df,  # pandas dataframe of data to validate model on
    target="Survived",
)

In [21]:
data_context

DataContext(name='my_dataset', version='v0.0.1', testing_data=        Sex Embarked   Title  Pclass   Age  SibSp  Parch   Fare  Survived
751    male        S  Master       3   6.0      0      1  12.48         1
90     male        S      Mr       3  29.0      0      0   8.05         0
852  female        C    Miss       3   9.0      1      1  15.25         0
718    male        Q      Mr       3   NaN      0      0  15.50         0
36     male        C      Mr       3   NaN      0      0   7.23         1
..      ...      ...     ...     ...   ...    ...    ...    ...       ...
879  female        C     Mrs       1  56.0      0      1  83.16         1
682    male        S      Mr       3  20.0      0      0   9.22         0
509    male        S      Mr       3  26.0      0      0  56.50         1
224    male        S      Mr       1  38.0      1      0  90.00         1
386    male        S  Master       3   1.0      5      2  46.90         0

[295 rows x 9 columns], target='Survived', traini

In [24]:
# 2. Build validations with ModelValidator
model_validator = ModelValidator(data=data_context, model=model)

In [25]:
validations = [
    model_validator.validate_performance_against_threshold(metric="accuracy", threshold=0.7),
    model_validator.validate_feature_in_top_n_important_features(feature="Age", top_n_features=3),
]

2023-05-24 17:49:33.701 | DEBUG    | trubrics.validations.model.base:_compute_permutation_feature_importance:629 - Computing permutation feature importance for testing_data with 10 permutations.


In [29]:
validations[0].dict()

{'validation_type': 'validate_performance_against_threshold',
 'validation_kwargs': {'args': (),
  'kwargs': {'metric': 'accuracy', 'threshold': 0.7}},
 'passed': True,
 'severity': 'error',
 'result': {'performance': 0.7966101694915254, 'sample_size': 295},
 'explanation': '**Performance validation versus a fixed threshold value.**\n\nCompares performance of a model on any of the datasets in the DataContext to a hard coded threshold value.\n\nExample:\n```py\nfrom trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, model=model)\nmodel_validator.validate_performance_against_threshold(\nmetric="recall",\nthreshold=0.8\n)\n```\n\nArgs:\nmetric: performance metric name defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer fed in when initialising the ModelValidator object.\nthreshold: the performance threshold that the model must attain.\ndataset: the name of a dataset from the DataContext {\'testing_data\', \'training_data\'}.\ndat

In [30]:
# 3. Group validations into a Trubric
trubric = Trubric(
    name="my_first_trubric",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    validations=validations,
)

In [31]:
trubric

Trubric(name='my_first_trubric', passed=None, total_passed=None, total_failed=None, failing_severity='error', data_context_name='my_dataset', data_context_version='v0.0.1', model_name=None, model_version=None, tags=[], run_by=None, git_commit=None, timestamp=None, metadata=None, validations=[Validation(validation_type='validate_performance_against_threshold', validation_kwargs={'args': (), 'kwargs': {'metric': 'accuracy', 'threshold': 0.7}}, passed=True, severity='error', result={'performance': 0.7966101694915254, 'sample_size': 295}, explanation='**Performance validation versus a fixed threshold value.**\n\nCompares performance of a model on any of the datasets in the DataContext to a hard coded threshold value.\n\nExample:\n```py\nfrom trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, model=model)\nmodel_validator.validate_performance_against_threshold(\nmetric="recall",\nthreshold=0.8\n)\n```\n\nArgs:\nmetric: performance metric name def

In [36]:
trubric.save_local(path='.')  # save trubric as a local .json file
# trubric.save_ui()  # or to the Trubrics platform

ValueError: Reference at 'refs/heads/main' does not exist